In [12]:
import pandas as pd
import json
import numpy as np

from itertools import product
from tqdm.notebook import tqdm

In [13]:
with open('telco_directed.json') as f:
    data= json.load(f)

In [14]:
#edge_list
location_list = list(product(data['locations'],data['locations']))

values = np.array(data['data'])

df = pd.DataFrame()
df['src'] = [i[0] for i in location_list]
df['dst'] = [i[1] for i in location_list]

for idx,date in enumerate(data['dates']):
    
    df[date] = values[:,idx]

In [15]:
df = df.melt(id_vars=['src','dst'],var_name='day')

In [16]:
df

,src,dst,day,value
0,Aabenraa,Aabenraa,2020-02-01 00:00:00,323162.099102
1,Aabenraa,Aalborg,2020-02-01 00:00:00,405.268442
2,Aabenraa,Albertslund,2020-02-01 00:00:00,0.000000
3,Aabenraa,Allerød,2020-02-01 00:00:00,0.000000
4,Aabenraa,Assens,2020-02-01 00:00:00,690.133217
...,...,...,...,...
4955659,Ærø,Vesthimmerland,2021-06-30 00:00:00,0.000000
4955660,Ærø,Viborg,2021-06-30 00:00:00,0.000000
4955661,Ærø,Vordingborg,2021-06-30 00:00:00,0.000000
4955662,Ærø,Århus,2021-06-30 00:00:00,0.000000


In [9]:
df['src'].nunique()

98

In [10]:
list(data.keys())

['data', 'locations', 'dates']

In [11]:
len(data['dates'])

516